### https://techblog-history-younghunjo1.tistory.com/376
- 신경망 모델은 덧셈, 곱셈 연산도 존재하지만 가장 핵심은 Activation Function Operation 이다.
- 활성화 함수 종류에 따라 오차역전파 방법이 차이가 있다.

# 1. ReLU
- ReLU는 x가 0보다 작거나 같을 때, y를 모두 0으로 차단해버리는 활성화함수 이다.
- ReLU 함수의 x에 대한 y의 미분은 순전파 때의 입력 데이터가 0보다 컸다면, 이를 역전파 수행할 때 이전 노드에서 흘러들어온 국소적인 미분 값을 그대로 흘려보내는 것을 의미한다.
- 반대로 순전파 시 x가 0이었다면 이를 역전파 수행 시, 이전 노드에서 흘러들어온 국소적인 미분 값을 전달하지 않는 것을 의미한다. x가 0 이하일 때 미분값이 0이기 때문에 이전 노드에서 어떤 값이 흘러나왔던 간에 0을 곱하면 0이 되기 때문이다.

In [22]:
import numpy as np

# Relu 활성함수의 역전파 계층 만들기
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x: np.array):
        # 0보다 작거나 같은 값은 출력값 0처리,
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, d_out):
        d_out[self.mask] = 0
        dx = d_out
        
        return dx
    
datas = np.append(np.zeros(10), np.ones(10) + 1)
np.random.shuffle(datas)
print("Input - {}".format(datas))

relu_layer = Relu()
forward_result = relu_layer.forward(datas)
print("Forward Propagation - {}".format(forward_result))

back_result = relu_layer.backward(datas)
print("Backward Propagation - {}".format(back_result))

Input - [2. 2. 0. 2. 2. 2. 0. 2. 0. 0. 0. 2. 0. 2. 0. 0. 0. 2. 0. 2.]
Forward Propagation - [2. 2. 0. 2. 2. 2. 0. 2. 0. 0. 0. 2. 0. 2. 0. 0. 0. 2. 0. 2.]
Backward PRopagation - [2. 2. 0. 2. 2. 2. 0. 2. 0. 0. 0. 2. 0. 2. 0. 0. 0. 2. 0. 2.]


# 2. Sigmoid
- 시그모이드 함수 연산은 계산 그래프(Computation Graph)로 펼쳐서 확인할 수 있다.
1. 나누기 연산
2. 덧셈 연산
3. exp(Exponential) 연산

In [13]:
def Sigmoid:
    def __init__:
        

array([1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 1.])